In [1]:
#| default_exp features

In [2]:
#| hide
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

# Feature Engineering
> In this notebook, our aim is to extract and compute the features required for the model.

The features that are available for a particular point in time are exclusively derived from data related to events happening during play, including the location of the event, the player in possession of the ball, the time of the event, and additional information like the current score. However, when analyzing the context of a player not in possession of the ball, we need to consider their spatiotemporal situation, which includes the time when their location was last observed, i.e., when they were last in possession of the ball. These features are calculated for each player, and they include both player-specific and global features. Specifically, for a particular moment in time (denoted as $t$) with a corresponding event (denoted as e𝑡 ∈ $E$), and an observed player (denoted as $a_{n}$ ∈ $A$), we compute the following set of features:

## Agent-Specific Features
* **prevAgentTime, prevAgentX, prevAgentY**: Last time agent was observed doing an event and its location at the time.
* **nextAgentTime, nextAgentX, nextAgentY**: Next time agent is observed doing an event and its location at the time.
* **avAgentX, avAgentY**: mean location of the agent across the entire game.
* **agentRole**: Agent role in team.
* **agentSide**: binary indicator of whether an agent is on the same team as the current ball-carrying agent.
* **agentObserved**: binary indicator of whether the agent is observed or not.
* **goalDiff**: the difference in score between the teams at each timestep.

## Global Features
* **eventX,eventY**: The coordinates of the event $e_{x,y}$.
* **eventType**: The type of event.

In [3]:
#| export

import itertools
import json
import math
import os
import re
from pathlib import Path
from typing import Callable, List, Optional, Tuple

import numpy as np
import pandas as pd
import torch

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler

from agent_imputter.utils import *

In [4]:
# to display scientific notation as float for debugging
np.set_printoptions(suppress=True)

## Load Data

For the Data we require using these files :
* `optical_tracking_df.csv`
* `optical_opta_mapping.csv`
* `opta_events_reduced.csv`
* `lineup.csv`
* `ins_outs.csv`

In [5]:
event_provider = "opta"
tracking_provider = "secondSpectrum"

In [6]:
game_path = Path(
    "/home/user/Downloads/Data/games/1059704_2019-08-10_Burnley_Southampton"
)
game_id = 1059704
tracking_provider = "secondSpectrum"
events_df, tracking_df, lineup_df = get_game_dataframes(
    game_path=game_path,
    game_id=game_id,
    event_provider=event_provider,
    tracking_provider=tracking_provider,
)

### Functions

In [7]:
#| export
def get_tracking_indexes(
    events_df: pd.DataFrame(), tracking_df: pd.DataFrame()
) -> np.array:
    "get tracking indexes related to each event"
    return events_df["frameId"].values

In [8]:
tracking_indexes = get_tracking_indexes(events_df, tracking_df)

## Player numerical features

In [9]:
#| export
def _get_player_events_half(
    half_events_df: pd.DataFrame(),
    player: str,
) -> pd.DataFrame():
    "get player numerical features of one half"
    ball_x = half_events_df["tr_x"]
    ball_y = half_events_df["tr_y"]
    time_since_last_pred = half_events_df["event_secs"].diff().fillna(0)
    prev_player_time = []
    next_player_time = []
    l_prev_player_x = []
    l_prev_player_y = []
    l_next_player_x = []
    l_next_player_y = []

    for i in range(half_events_df.index[0], half_events_df.index[-1] + 1):
        prev_events = half_events_df.loc[: i - 1]
        next_events = half_events_df.loc[i + 1 :]
        prev_player_event = prev_events[
            (prev_events["playerKey"] == player)
            | (prev_events["opponentKey"] == player)
        ].tail(1)
        next_player_event = next_events[
            (next_events["playerKey"] == player)
            | (next_events["opponentKey"] == player)
        ].head(1)

        if len(prev_player_event) == 0:
            l_prev_player_x.append(0)
            l_prev_player_y.append(0)
            prev_player_time.append(1000000)
        elif len(prev_player_event) == 1:
            l_prev_player_x.append(prev_player_event["tr_x"].values[0])
            l_prev_player_y.append(prev_player_event["tr_y"].values[0])
            last_player_time = (
                half_events_df.iloc[i]["event_secs"]
                - prev_player_event["event_secs"].values[0]
            )
            prev_player_time.append(last_player_time)

        if len(next_player_event) == 0:
            l_next_player_x.append(0)
            l_next_player_y.append(0)
            next_player_time.append(1000000)
        elif len(next_player_event) == 1:
            l_next_player_x.append(next_player_event["tr_x"].values[0])
            l_next_player_y.append(next_player_event["tr_y"].values[0])
            later_player_time = (
                next_player_event["event_secs"].values[0]
                - half_events_df.iloc[i]["event_secs"]
            )
            next_player_time.append(later_player_time)

    return pd.DataFrame(
        {
            "ballx": ball_x,
            "bally": ball_y,
            "time_since_last_pred": time_since_last_pred,
            "prev_player_x": l_prev_player_x,
            "prev_player_y": l_prev_player_y,
            "prev_player_time": prev_player_time,
            "next_player_x": l_next_player_x,
            "next_player_y": l_next_player_y,
            "next_player_time": next_player_time,
        }
    )


def get_player_events_data(
    events_df: pd.DataFrame(),
    player: str,
) -> pd.DataFrame():
    "get player numerical features"
    events_FH = events_df[events_df["period_id"] == 1].copy().reset_index(drop=True)
    events_SH = events_df[events_df["period_id"] == 2].copy().reset_index(drop=True)
    info_events_FH = _get_player_events_half(events_FH, player)
    info_events_SH = _get_player_events_half(events_SH, player)

    return pd.concat([info_events_FH, info_events_SH]).reset_index(drop=True)

In [10]:
player = "home_player_4"
pl_num_feats = get_player_events_data(events_df, player)

## Player categorical features

In [11]:
#| export
def get_player_categorical_data(
    events_df: pd.DataFrame(),
    player: str,
    team_df: pd.DataFrame(),
) -> pd.DataFrame():
    "get player categorical features"
    side = player.split("_")[0]
    jersey_number = int(player.split("_")[2])
    position = team_df[
        (team_df["side"] == side) & (team_df["jerseyNumber"] == jersey_number)
    ]["position_sportLogic"].values[0]
    event_type = events_df["event_name"]
    event_id = events_df["event_key"]
    match_id = events_df["gameId"]
    event_time = events_df["event_secs"]
    period = events_df["period_id"]
    team_on_ball = events_df["playerKey"].isin(
        [
            f"{_side}_player_{_jersey}"
            for _side, _jersey in zip(team_df["side"], team_df["jerseyNumber"])
        ]
    )
    player_on_ball = pd.Series(np.where(events_df["playerKey"] == player, True, False))
    return pd.DataFrame(
        {
            "event_id": event_id,
            "match_id": match_id,
            "event_time": event_time,
            "period_id": period,
            "player_id": player,
            "position": position,
            "event_type": event_type,
            "team_on_ball": team_on_ball,
            "player_on_ball": player_on_ball,
        }
    )

In [12]:
pl_cat_feats = get_player_categorical_data(
    events_df, "home_player_4", lineup_df[lineup_df["side"] == "home"]
).head(10)

## Score Difference

In [13]:
#| export
def get_goal_diff(events_df: pd.DataFrame(), team_id: int):
    "Calculate the goal difference at moments in a game"
    gc = (
        (events_df["event_name"].str.contains("Goal"))
        & (events_df["possessionTeamId"] == team_id)
    ).cumsum()
    gs = (
        (events_df["event_name"].str.contains("Goal"))
        & (events_df["possessionTeamId"] != team_id)
    ).cumsum()
    return gs - gc

In [14]:
gd = get_goal_diff(events_df, get_teams(events_df)[1])

In [15]:
#| export
def get_data(
    events_df: pd.DataFrame(),
    tracking_df: pd.DataFrame(),
    player: str,  # player key
    team_df: pd.DataFrame(),  # lineup team df
    team_gk: int,  # gk jersey
    tracking_indexes,
    goal_diff,  # team goal diff
    side: str,  # home or away
) -> Tuple:
    "get player features"
    ltr_mask_x = np.where(
        tracking_df[f"{side}_player_{team_gk}_x"][tracking_indexes].reset_index(
            drop=True
        )
        > 52.5,
        105,
        0,
    )

    # flip coords in events
    events_df["tr_x"] = abs(events_df[["tr_x"]] - ltr_mask_x[:, None])
    input_data = get_player_events_data(events_df, player)

    cats = get_player_categorical_data(events_df, player, team_df)
    cats = pd.concat([cats, goal_diff], axis=1)

    input_data = pd.concat(
        [input_data, cats], axis=1
    )  # torch.cat((input_x_data, input_y_data,cats),1)
    input_data["av_player_x"] = np.mean(
        input_data[input_data["player_on_ball"] == True]["ballx"]
    )
    input_data["av_player_y"] = np.mean(
        input_data[input_data["player_on_ball"] == True]["bally"]
    )

    label_x = np.array(tracking_df[f"{player}_x"][tracking_indexes])
    label_x = np.absolute(label_x - ltr_mask_x)
    label_y = np.array(tracking_df[f"{player}_y"][tracking_indexes])
    label_data = np.array(list(zip(label_x, label_y)))

    nan_rows = np.where((np.isnan(label_data)).all(axis=1))[0]
    label_data = np.delete(label_data, nan_rows, axis=0)
    input_data.drop(input_data.index[nan_rows], inplace=True)

    return input_data, label_data

In [16]:
_input_data, _label_data = get_data(
    events_df,
    tracking_df,
    "home_player_10",
    lineup_df[lineup_df["side"] == "home"],
    get_goalkeepers(lineup_df)[0],
    tracking_indexes,
    gd,
    "home",
)

In [17]:
#| export
def get_game_data(
    events_df: pd.DataFrame(),
    tracking_df: pd.DataFrame(),
    lineup_df: pd.DataFrame(),
    goalkeepers: Tuple,
) -> Tuple:
    "Get full feature set data for one game"
    tracking_indexes = get_tracking_indexes(events_df, tracking_df)
    whole_input = pd.DataFrame()
    whole_label = np.empty((0, 2), float)
    home_team_id, away_team_id = get_teams(events_df)

    home_team_df = lineup_df[lineup_df["side"] == "home"]
    away_team_df = lineup_df[lineup_df["side"] == "away"]

    home_goal_diff = pd.Series(get_goal_diff(events_df, home_team_id), name="goal_diff")
    away_goal_diff = pd.Series(get_goal_diff(events_df, away_team_id), name="goal_diff")

    home_player_key = [
        player_key[: player_key.find("_x")]
        for player_key in tracking_df.columns.tolist()
        if player_key.startswith("home") & player_key.endswith("_x")
    ]
    away_jersey_key = [
        player_key[: player_key.find("_x")]
        for player_key in tracking_df.columns.tolist()
        if player_key.startswith("away") & player_key.endswith("_x")
    ]

    for pl_key in home_player_key:
        input_data, label_data = get_data(
            events_df,
            tracking_df,
            pl_key,
            home_team_df,
            goalkeepers[0],
            tracking_indexes,
            home_goal_diff,
            "home",
        )
        whole_input = pd.concat([whole_input, input_data])
        whole_label = np.append(whole_label, label_data, axis=0)

    for pl_key in away_jersey_key:
        input_data, label_data = get_data(
            events_df,
            tracking_df,
            pl_key,
            away_team_df,
            goalkeepers[1],
            tracking_indexes,
            away_goal_diff,
            "away",
        )
        whole_input = pd.concat([whole_input, input_data])
        whole_label = np.append(whole_label, label_data, axis=0)

    # sort df
    whole_input["label_x"] = whole_label[:, 0]
    whole_input["label_y"] = whole_label[:, 1]
    whole_input = whole_input.sort_values(["period_id", "event_time"])
    # whole_label = torch.tensor(np.column_stack((whole_input["label_x"].values,whole_input["label_y"].values)))
    # whole_input = whole_input.drop(columns=["label_x", "label_y"])

    return whole_input

In [18]:
input_data = get_game_data(
    events_df, tracking_df, lineup_df, get_goalkeepers(lineup_df)
)

## Sort features DataFrame

In [19]:
#| export
def custom_sort(
    input_data: pd.DataFrame(),
) -> Tuple:
    "Sort the input data by event, and then team, and then position."

    position_dict = {
        "G": 0,
        "LFB": 1,
        "RFB": 2,
        "CMF": 3,
        "DMF": 4,
        "AMF": 5,
        "CD": 6,
        "S": 7,
        "SS": 8,
        "RW": 9,
        "LW": 10,
        "AMC": 11,
    }
    whole_df = pd.DataFrame()
    for e in input_data["event_id"].unique():
        events = input_data[(input_data["event_id"] == e)]
        if len(events) != 22:
            continue
        team_on_ball = events[(events["team_on_ball"] == True)]
        team_off_ball = events[(events["team_on_ball"] == False)]
        team_on_ball = team_on_ball.sort_values(
            by="position", key=lambda x: x.map(position_dict)
        )
        team_off_ball = team_off_ball.sort_values(
            by="position", key=lambda x: x.map(position_dict)
        )
        on_off = pd.concat([team_on_ball, team_off_ball])

        home_players = events[
            events["player_id"].str.startswith("home_player")
        ].sort_values(by="position", key=lambda x: x.map(position_dict))
        away_players = events[
            events["player_id"].str.startswith("away_player")
        ].sort_values(by="position", key=lambda x: x.map(position_dict))
        players = pd.concat([home_players, away_players])

        whole_df = pd.concat([whole_df, players])

    whole_df = whole_df.reset_index(drop=True)
    # whole_label = torch.tensor(np.column_stack((whole_df["label_x"].values,whole_df["label_y"].values)))
    # whole_df = whole_df.drop(columns=["label_x", "label_y"])

    event_num = pd.Index(range(int(len(whole_df) / 22))).repeat(22)
    whole_df.insert(0, "event_num", event_num)

    return whole_df

In [20]:
sorted_whole_input_df = custom_sort(input_data)

## Compute features for one game

In [21]:
#| export


def get_features_game(
    events_df: pd.DataFrame(), tracking_df: pd.DataFrame(), lineup_df: pd.DataFrame()
) -> Tuple:
    "Compute features and labels for one game"
    input_data = get_game_data(
        events_df, tracking_df, lineup_df, get_goalkeepers(lineup_df)
    )
    sorted_whole_input_df = custom_sort(input_data)
    return sorted_whole_input_df

In [22]:
features_df = get_features_game(events_df, tracking_df, lineup_df)

## Compute features for all games

In [23]:
#| export


def get_features_all_game(
    games_path: Path,
    tracking_provider: str,
    event_provider: str = "opta",
    save_df: bool = False,
) -> pd.DataFrame():
    "Compute features and labels for all games in dataset"

    match_folder = tuple(
        name
        for name in os.listdir(games_path)
        if os.path.isdir(os.path.join(games_path, name))
    )
    match_ids = [int(_folder.split("_")[0]) for _folder in match_folder]

    events_dfs = []
    tracking_dfs = []
    lineup_dfs = []
    num_games = len(match_ids)
    count = 0

    for _folder, match_id in zip(match_folder, match_ids):
        print(f"match id : {match_id}")
        _match_folder = Path(f"{games_path}{_folder}")
        events_df, tracking_df, lineup_df = get_game_dataframes(
            _match_folder, match_id, event_provider, tracking_provider
        )
        count += len(events_df)
        events_dfs.append(events_df)
        tracking_dfs.append(tracking_df)
        lineup_dfs.append(lineup_df)

    whole_input = []

    for i, _folder, match_id in zip(range(0, num_games), match_folder, match_ids):
        print(f"game num : {i}")
        input_data = get_features_game(events_dfs[i], tracking_dfs[i], lineup_dfs[i])
        whole_input.append(input_data)
        if save_df:
            _match_folder = Path(f"{games_path}{_folder}")
            input_data.to_csv(f"{_match_folder}/features.csv")
    return whole_input

In [ ]:
# |eval: false

games_path = "/home/user/Downloads/Data/games/"
tracking_provider = "secondSpectrum"

df = get_features_all_game(games_path, tracking_provider, save_df=True)

match id : 2128648
match id : 2210383
match id : 2210593
match id : 1059830
match id : 2128632
match id : 1059712
match id : 1059763
match id : 2128326
match id : 2128614
match id : 1059894
match id : 2210567
match id : 1059733
match id : 1059704
match id : 2210573
match id : 1059744
match id : 2210589
match id : 2210534
match id : 1059753
match id : 2210513
match id : 2128618
match id : 2210605
match id : 2210554
match id : 2210542
match id : 2210612
match id : 2210524
match id : 1059914
match id : 2128358
match id : 2128294
match id : 2128342
match id : 2128641
match id : 2128413
match id : 1059787
match id : 2128318
match id : 1059832


In [ ]:
#| hide
from nbdev import nbdev_export

nbdev_export()